In [2]:
import gym_super_mario_bros
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT #Importing simple controls

In [3]:
import gym_super_mario_bros
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT #Importing simple controls

In [5]:
#Creating the environment
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)

In [6]:
#Random action
SIMPLE_MOVEMENT[env.action_space.sample()]

['right']

In [7]:
#Space shape
print(env.observation_space.shape)

(240, 256, 3)


In [9]:
#Testing the gym library
done = True
for step in range(10000):
    if done:
        state = env.reset()
    state, reward, done, info = env.step(env.action_space.sample()) #Random movements
    env.render()

env.close()

C:\Users\evely\anaconda3\envs\entornoTFGgpu\lib\site-packages\pyglet\image\codecs\wic.py:289: UserWarning: [WinError -2147417850] No se puede cambiar el modo de subproceso después de establecerlo
  warnings.warn(str(err))
